In [1]:
import pandas as pd
import os

# --- Configuration: UPDATE THESE PATHS AND COLUMN NAMES ---
# 1. Replace 'your_simple_data.csv' with the actual filename of your simpler excel file
#    (the one with SAMPLE_ID in Column A).
simple_data_file = "C:/Users/Admin/Desktop/Combined_Document_With_Filenames.csv"

# 2. Replace 'metadata.csv' with the actual filename of your metadata file.
metadata_file = "C:/Users/Admin/Desktop/metadata.cart.2025-06-09.csv"

# 3. Replace 'SAMPLE_ID_COLUMN_IN_SIMPLE_DATA' with the actual column name for SAMPLE_ID in your 'simple_data_file'.
#    (e.g., if Column A is named 'Sample_ID', use 'Sample_ID')
sample_id_col_simple_data = 'SAMPLE_ID' # Example: 'SAMPLE_ID' or 'Column A'

# 4. Replace 'SAMPLE_ID_COLUMN_IN_METADATA' with the actual column name for Column C in your 'metadata_file'.
#    (e.g., if Column C is named 'sample_id', use 'sample_id')
sample_id_col_metadata = 'associated_entities/0/entity_submitter_id' # Example: 'sample_id' or 'C'

# 5. Replace 'FILE_NAME_PATH_COLUMN_IN_METADATA' with the actual column name for Column G in your 'metadata_file'.
#    (e.g., if Column G is named 'file_path_gdc', use 'file_path_gdc')
file_name_path_col_metadata = 'file_name' # Example: 'file_path_gdc' or 'G'

# 6. Define the suffix to remove from the file path to get the desired 'File_name'.
file_suffix_to_remove = '.rna_seq.augmented_star_gene_counts.tsv'

# 7. Define the name for the output file
output_file_name = 'enriched_simple_data.csv'

# --- Load Data ---
try:
    df_simple = pd.read_csv(simple_data_file)
    print(f"Loaded '{simple_data_file}' successfully.")
    print("\n--- Preview of your simple data ---")
    print(df_simple.head())
    print("\n--- Info for your simple data ---")
    df_simple.info()
    print("-" * 30)

    df_metadata = pd.read_csv(metadata_file)
    print(f"\nLoaded '{metadata_file}' successfully.")
    print("\n--- Preview of metadata ---")
    print(df_metadata.head())
    print("\n--- Info for metadata ---")
    df_metadata.info()
    print("-" * 30)

except FileNotFoundError as e:
    print(f"Error: One of the specified files was not found. Please check filenames and paths. {e}")
    exit()
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    exit()

# --- Data Processing ---

# 1. Extract 'File_name' from metadata
if file_name_path_col_metadata in df_metadata.columns:
    df_metadata['File_name'] = df_metadata[file_name_path_col_metadata].astype(str).apply(
        lambda x: x.replace(file_suffix_to_remove, '') if file_suffix_to_remove in x else x
    )
    print(f"\nSuccessfully extracted 'File_name' from '{file_name_path_col_metadata}' column.")
else:
    print(f"\nError: Column '{file_name_path_col_metadata}' not found in metadata file. Please check your configuration.")
    exit()

# Select only relevant columns from metadata for merging
df_metadata_for_merge = df_metadata[[sample_id_col_metadata, 'File_name']].copy()

# 2. Merge the dataframes
if sample_id_col_simple_data in df_simple.columns and sample_id_col_metadata in df_metadata_for_merge.columns:
    # Rename metadata's sample ID column to match simple data's for direct merge
    df_metadata_for_merge.rename(columns={sample_id_col_metadata: sample_id_col_simple_data}, inplace=True)

    merged_df = pd.merge(
        df_simple,
        df_metadata_for_merge,
        on=sample_id_col_simple_data,
        how='left' # Use 'left' merge to keep all rows from your simple data
    )
    print("\nDataframes merged successfully.")
else:
    print(f"\nError: Missing sample ID column in one of the dataframes. Please check '{sample_id_col_simple_data}' and '{sample_id_col_metadata}'.")
    exit()

# --- Save Results ---
try:
    merged_df.to_csv(output_file_name, index=False)
    print(f"\nSuccessfully saved the enriched data to '{output_file_name}'")
    print("\n--- Preview of the enriched data ---")
    print(merged_df.head())
    print(f"\nOutput file saved to: {os.path.abspath(output_file_name)}")
except Exception as e:
    print(f"Error saving the output file: {e}")

Loaded 'C:/Users/Admin/Desktop/Combined_Document_With_Filenames.csv' successfully.

--- Preview of your simple data ---
         SAMPLE_ID  Unnamed: 1    PATIENT_ID  Unnamed: 3 Subtype  AR_score  \
0  TCGA-HC-7818-01         NaN  TCGA-HC-7818         NaN   1-ERG      1.04   
1  TCGA-HC-7077-01         NaN  TCGA-HC-7077         NaN   1-ERG     -2.72   
2  TCGA-G9-6356-01         NaN  TCGA-G9-6356         NaN   1-ERG    -10.25   
3  TCGA-HC-7213-01         NaN  TCGA-HC-7213         NaN   1-ERG     -4.25   
4  TCGA-KK-A6E1-01         NaN  TCGA-KK-A6E1         NaN   1-ERG     -1.27   

   associated_entities/0/case_id  File_name  
0                            NaN        NaN  
1                            NaN        NaN  
2                            NaN        NaN  
3                            NaN        NaN  
4                            NaN        NaN  

--- Info for your simple data ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 8 column